In [1]:
import numpy as np
import pandas as pd

In [2]:
__author__ = 'Chris Potts'


class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        The temperature parameter. Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        return rownorm(self.lexicon * self.prior)

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        spk = self.speaker().T
        return rownorm(spk * self.prior)


def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T


def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)


if __name__ == '__main__':
    """Examples"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['шляпа', 'очки', 'усы']
    states = ['r1', 'r2','r3']
    lex = pd.DataFrame([
        [1.0, 1.0, 0.0],
        [0.0, 0.0, 1.0],
        [0.0, 1.0, 0.0]], index=msgs, columns=states)

In [3]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0.0, 0.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
шляпа,1.000000,1.0,0.0,0.0
очки,0.000000,0.0,1.0,0.0
усы,0.000000,1.0,0.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Literal listener


,r1,r2,r3
шляпа,0.5,0.5,0.0
очки,0.0,0.0,1.0
усы,0.0,1.0,0.0



Pragmatic speaker


,шляпа,очки,усы
r1,1.000000,0.0,0.000000
r2,0.333333,0.0,0.666667
r3,0.000000,1.0,0.000000



Pragmatic listener


,r1,r2,r3
шляпа,0.75,0.25,0.0
очки,0.00,0.00,1.0
усы,0.00,1.00,0.0


In [4]:
    print("="*70 + "\nEven priors, imbalanced message costs\n")
    cost_most = RSA(lexicon=lex,  prior=[0.1, 0.45, 0.45], costs=[0.0, 0.0, 0.0])

    display_reference_game(cost_most)

    print("\nLiteral listener")
    display(cost_most.literal_listener())

    print("\nPragmatic speaker")
    display(cost_most.speaker())

    print("\nPragmatic listener")
    display(cost_most.listener())

Even priors, imbalanced message costs



,r1,r2,r3,costs
шляпа,1.0,1.0,0.0,0.0
очки,0.0,0.0,1.0,0.0
усы,0.0,1.0,0.0,0.0
prior,0.1,0.45,0.45,
alpha,1.0,,,



Literal listener


,r1,r2,r3
шляпа,0.181818,0.818182,0.0
очки,0.000000,0.000000,1.0
усы,0.000000,1.000000,0.0



Pragmatic speaker


,шляпа,очки,усы
r1,1.00,0.0,0.00
r2,0.45,0.0,0.55
r3,0.00,1.0,0.00



Pragmatic listener


,r1,r2,r3
шляпа,0.330579,0.669421,0.0
очки,0.000000,0.000000,1.0
усы,0.000000,1.000000,0.0


In [5]:
msgs = ['нейтральный', 'слезы', 'улыбка', 'грусть']
states = ['r1', 'r2','r3','r4']
lex = pd.DataFrame([
        [1.0, 0.0, 0.0, 0.0],
        [0.0, 1.0, 1.0, 0.0],
        [0.0, 0.0, 1.0, 1.0],
        [0.0, 1.0, 0.0, 0.0]], index=msgs, columns=states)

print("="*70 + "\nEven priors and all-0 message costs\n")
basic_mod = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.0, 0.0, 0.0, 0.0])

display_reference_game(basic_mod)

print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,r4,costs
нейтральный,1.00,0.0,0.0,0.0,0.0
слезы,0.00,1.0,1.0,0.0,0.0
улыбка,0.00,0.0,1.0,1.0,0.0
грусть,0.00,1.0,0.0,0.0,0.0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Literal listener


,r1,r2,r3,r4
нейтральный,1.0,0.0,0.0,0.0
слезы,0.0,0.5,0.5,0.0
улыбка,0.0,0.0,0.5,0.5
грусть,0.0,1.0,0.0,0.0



Pragmatic speaker


,нейтральный,слезы,улыбка,грусть
r1,1.0,0.000000,0.0,0.000000
r2,0.0,0.333333,0.0,0.666667
r3,0.0,0.500000,0.5,0.000000
r4,0.0,0.000000,1.0,0.000000



Pragmatic listener


,r1,r2,r3,r4
нейтральный,1.0,0.0,0.000000,0.000000
слезы,0.0,0.4,0.600000,0.000000
улыбка,0.0,0.0,0.333333,0.666667
грусть,0.0,1.0,0.000000,0.000000


In [6]:
#посмотрим, как меняются вероятности для прагматического слушающего с изменением параметра alpha
print("="*70 + "\nEven priors and all-0 message costs; alpha = 2\n")
alpha_mod = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.0, 0.0, 0.0, 0.0], alpha=2.0)

display_reference_game(alpha_mod)

print("\nLiteral listener")
display(alpha_mod.literal_listener())

print("\nPragmatic speaker")
display(alpha_mod.speaker())

print("\nPragmatic listener")
display(alpha_mod.listener())

Even priors and all-0 message costs; alpha = 2



,r1,r2,r3,r4,costs
нейтральный,1.00,0.0,0.0,0.0,0.0
слезы,0.00,1.0,1.0,0.0,0.0
улыбка,0.00,0.0,1.0,1.0,0.0
грусть,0.00,1.0,0.0,0.0,0.0
prior,0.25,0.25,0.25,0.25,
alpha,2.00,,,,



Literal listener


,r1,r2,r3,r4
нейтральный,1.0,0.0,0.0,0.0
слезы,0.0,0.5,0.5,0.0
улыбка,0.0,0.0,0.5,0.5
грусть,0.0,1.0,0.0,0.0



Pragmatic speaker


,нейтральный,слезы,улыбка,грусть
r1,1.0,0.0,0.0,0.0
r2,0.0,0.2,0.0,0.8
r3,0.0,0.5,0.5,0.0
r4,0.0,0.0,1.0,0.0



Pragmatic listener


,r1,r2,r3,r4
нейтральный,1.0,0.000000,0.000000,0.000000
слезы,0.0,0.285714,0.714286,0.000000
улыбка,0.0,0.000000,0.333333,0.666667
грусть,0.0,1.000000,0.000000,0.000000


In [7]:
#меняем параметр цены высказывания
print("="*70 + "\nEven priors, imbalanced message costs\n")
cost_most = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[-3.0, -2.0, 0.0, 1.0])

display_reference_game(cost_most)

print("\nLiteral listener")
display(cost_most.literal_listener())

print("\nPragmatic speaker")
display(cost_most.speaker())

print("\nPragmatic listener")
display(cost_most.listener())

Even priors, imbalanced message costs



,r1,r2,r3,r4,costs
нейтральный,1.00,0.0,0.0,0.0,-3.0
слезы,0.00,1.0,1.0,0.0,-2.0
улыбка,0.00,0.0,1.0,1.0,0.0
грусть,0.00,1.0,0.0,0.0,1.0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Literal listener


,r1,r2,r3,r4
нейтральный,1.0,0.0,0.0,0.0
слезы,0.0,0.5,0.5,0.0
улыбка,0.0,0.0,0.5,0.5
грусть,0.0,1.0,0.0,0.0



Pragmatic speaker


,нейтральный,слезы,улыбка,грусть
r1,1.0,0.000000,0.000000,0.000000
r2,0.0,0.024289,0.000000,0.975711
r3,0.0,0.119203,0.880797,0.000000
r4,0.0,0.000000,1.000000,0.000000



Pragmatic listener


,r1,r2,r3,r4
нейтральный,1.0,0.00000,0.000000,0.000000
слезы,0.0,0.16927,0.830730,0.000000
улыбка,0.0,0.00000,0.468311,0.531689
грусть,0.0,1.00000,0.000000,0.000000


In [8]:
#еще раз меняем параметр цены высказывания, но теперь только для "слезы"
print("="*70 + "\nEven priors, imbalanced message costs\n")
cost_most = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.0, 4.0, 0.0, 0.0])

display_reference_game(cost_most)

print("\nLiteral listener")
display(cost_most.literal_listener())

print("\nPragmatic speaker")
display(cost_most.speaker())

print("\nPragmatic listener")
display(cost_most.listener())

Even priors, imbalanced message costs



,r1,r2,r3,r4,costs
нейтральный,1.00,0.0,0.0,0.0,0.0
слезы,0.00,1.0,1.0,0.0,4.0
улыбка,0.00,0.0,1.0,1.0,0.0
грусть,0.00,1.0,0.0,0.0,0.0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Literal listener


,r1,r2,r3,r4
нейтральный,1.0,0.0,0.0,0.0
слезы,0.0,0.5,0.5,0.0
улыбка,0.0,0.0,0.5,0.5
грусть,0.0,1.0,0.0,0.0



Pragmatic speaker


,нейтральный,слезы,улыбка,грусть
r1,1.0,0.000000,0.000000,0.000000
r2,0.0,0.964663,0.000000,0.035337
r3,0.0,0.982014,0.017986,0.000000
r4,0.0,0.000000,1.000000,0.000000



Pragmatic listener


,r1,r2,r3,r4
нейтральный,1.0,0.000000,0.000000,0.000000
слезы,0.0,0.495544,0.504456,0.000000
улыбка,0.0,0.000000,0.017668,0.982332
грусть,0.0,1.000000,0.000000,0.000000


In [9]:
#изменяем исходные (априорные) вероятности объектов
print("="*70 + "\nImbalanced priors, all-0 message costs\n")
prior_mod = RSA(lexicon=lex, prior=[0.1, 0.6, 0.1, 0.1], costs=[0.0, 0.0, 0.0, 0.0])

display_reference_game(prior_mod)

print("\nLiteral listener")
display(prior_mod.literal_listener())

print("\nPragmatic speaker")
display(prior_mod.speaker())

print("\nPragmatic listener")
display(prior_mod.listener())

Imbalanced priors, all-0 message costs



,r1,r2,r3,r4,costs
нейтральный,1.0,0.0,0.0,0.0,0.0
слезы,0.0,1.0,1.0,0.0,0.0
улыбка,0.0,0.0,1.0,1.0,0.0
грусть,0.0,1.0,0.0,0.0,0.0
prior,0.1,0.6,0.1,0.1,
alpha,1.0,,,,



Literal listener


,r1,r2,r3,r4
нейтральный,1.0,0.000000,0.000000,0.0
слезы,0.0,0.857143,0.142857,0.0
улыбка,0.0,0.000000,0.500000,0.5
грусть,0.0,1.000000,0.000000,0.0



Pragmatic speaker


,нейтральный,слезы,улыбка,грусть
r1,1.0,0.000000,0.000000,0.000000
r2,0.0,0.461538,0.000000,0.538462
r3,0.0,0.222222,0.777778,0.000000
r4,0.0,0.000000,1.000000,0.000000



Pragmatic listener


,r1,r2,r3,r4
нейтральный,1.0,0.000000,0.000000,0.0000
слезы,0.0,0.925714,0.074286,0.0000
улыбка,0.0,0.000000,0.437500,0.5625
грусть,0.0,1.000000,0.000000,0.0000
